### Differentiation on Linear Subspaces of G

In [1]:
from kingdon import Algebra
from gc_utils import *
import numpy as np
alg = Algebra(1,3)
locals().update(alg.blades)

In [2]:
#2.1
A = random_multivector(alg)
B = random_multivector(alg)

A2 = random_r_blade(2, alg)
F = lambda X: P(X, A2)**2
X = random_multivector(alg)
h = 6e-6 # leave h as a global var, so to modify with ease

# central scheme, modify h to minimize truncation/round-off error
# a measure along a line
def differential(F, X, A):
    d = (h)*A
    return (1/(2*h))*F(X+d) - (1/(2*h))*F(X-d)

In [3]:
# a suggestion of h
def auto_step_size(x):
    """
    Automatically determine step size based on variable magnitude and machine epsilon.
    """
    eps = np.finfo(float).eps
    return np.maximum(np.abs(x) * np.cbrt(eps), 100 * eps)
X = random_multivector(alg)
np.mean(auto_step_size(X[:]))

3.208847607182058e-06

In [4]:
#2.7 linearity
G = lambda X: (X+A)**3
differential(lambda X: A*F(X) + G(X), X, B), A * differential(F, X, B) + differential(G,X,B)

(-94.5 + -1.03e+02 𝐞₁ + -79.4 𝐞₂ + -2.43 𝐞₃ + -24.0 𝐞₄ + -48.3 𝐞₁₂ + -3.41 𝐞₁₃ + -3.34 𝐞₁₄ + -20.4 𝐞₂₃ + -6.82 𝐞₂₄ + -26.3 𝐞₃₄ + -24.1 𝐞₁₂₃ + -32.1 𝐞₁₂₄ + -8.66 𝐞₁₃₄ + 12.1 𝐞₂₃₄ + -6.95 𝐞₁₂₃₄,
 -94.5 + -1.03e+02 𝐞₁ + -79.4 𝐞₂ + -2.43 𝐞₃ + -24.0 𝐞₄ + -48.3 𝐞₁₂ + -3.41 𝐞₁₃ + -3.34 𝐞₁₄ + -20.4 𝐞₂₃ + -6.82 𝐞₂₄ + -26.3 𝐞₃₄ + -24.1 𝐞₁₂₃ + -32.1 𝐞₁₂₄ + -8.66 𝐞₁₃₄ + 12.1 𝐞₂₃₄ + -6.95 𝐞₁₂₃₄)

In [5]:
#2.8 function product
differential(lambda X: F(X)*G(X), X, A), differential(F, X, A) * G(X) + F(X) * differential(G, X, A)

(4.48e+02 + 1.66e+02 𝐞₁ + 76.2 𝐞₂ + -33.8 𝐞₃ + 37.0 𝐞₄ + 3.13e+02 𝐞₁₂ + -5.87e+02 𝐞₁₃ + -7.64e+02 𝐞₁₄ + -6e+02 𝐞₂₃ + -7.6e+02 𝐞₂₄ + -2.7e+02 𝐞₃₄ + -2.09e+02 𝐞₁₂₃ + -66.3 𝐞₁₂₄ + -4.83e+02 𝐞₁₃₄ + -4.36e+02 𝐞₂₃₄ + -2.02e+02 𝐞₁₂₃₄,
 4.48e+02 + 1.66e+02 𝐞₁ + 76.2 𝐞₂ + -33.8 𝐞₃ + 37.0 𝐞₄ + 3.13e+02 𝐞₁₂ + -5.87e+02 𝐞₁₃ + -7.64e+02 𝐞₁₄ + -6e+02 𝐞₂₃ + -7.6e+02 𝐞₂₄ + -2.7e+02 𝐞₃₄ + -2.09e+02 𝐞₁₂₃ + -66.3 𝐞₁₂₄ + -4.83e+02 𝐞₁₃₄ + -4.36e+02 𝐞₂₃₄ + -2.02e+02 𝐞₁₂₃₄)

In [6]:
#2.9
GF = lambda X: G(F(X))
max_diff(differential(GF, X, A), differential(G, F(X), differential(F, X, A)))

2.60770320892334e-08

In [7]:
#2.12
B = random_multivector(alg)
FA = lambda X: differential(F, X, A)
FB = lambda X: differential(F, X, B)
FAB = differential(FA, X, B)
FBA = differential(FB, X, A)
max_diff(FAB, FBA)

1.1641532182693481e-10

In [8]:
# a measure around a point X, reciprocal frame assures each direction measured evenly
def derivative(F, X, alg, grade=None, frame=None, r_frame=None):
    if not frame:
        if grade or (grade == 0):
            frame = r_vector_frame(alg.frame, grade)
            r_frame = r_vector_frame(reciprocal(alg.frame), grade, reverse=True)
        else:
            frame = multi_frame(alg.frame)
            r_frame = reci_frame(alg.frame)
    dF = 0
    for v, r in zip(frame, r_frame):
        dF += r * (differential(F, X, v))
    return dF


In [9]:
# Can we recover differential from derivative? Yes, but in the sense that the derivative is invertible. That's integration!
# These two are not equal!
A.sp(derivative(F, X, alg)), differential(F, X, A)

(3.9,
 1.19 + 2.87 𝐞₁ + 2.61 𝐞₂ + 2.13 𝐞₃ + 0.618 𝐞₄ + -1.74 𝐞₁₂ + 5.47 𝐞₁₃ + 8.25 𝐞₁₄ + 6.26 𝐞₂₃ + 7.85 𝐞₂₄ + 4.94 𝐞₃₄ + -7.4e-11 𝐞₁₂₃ + -2.96e-10 𝐞₁₂₄ + -3.7e-11 𝐞₁₃₄ + 1.48e-10 𝐞₂₃₄ + 5.55e-10 𝐞₁₂₃₄)

In [11]:
# Because the scalar product is prior
frame = multi_frame(alg.frame)
r_frame = reci_frame(alg.frame)
dF = 0
for v, r in zip(frame, r_frame):
    dF += alg.sp(r, A) * (differential(F, X, v))
max_diff(differential(F, X, A), dF)

5.557592409409528e-10

In [59]:
# Yet we have a equality (similar to 2.19) for the scalar part
# The classical gradient
g = lambda X: G(X).grade(0)
A.sp(derivative(g, X, alg)), differential(g, X, A)

(-49.9, -49.9)

In [60]:
#2.13
# Randomly generate several frames to see dF is independent of frame
r = 4
dFs = []
for _ in range(5):
    basis = random_r_vectors(r, alg)
    r_basis = reciprocal(basis)
    frame = multi_frame(basis)
    r_frame = multi_frame(r_basis, reverse=True)

    dF = derivative(F, X, alg, frame=frame, r_frame=r_frame)
    dFs.append(dF)
[max_diff(dFs[-1], dF) for dF in dFs[:-1]]

[2.6945804303089626e-12,
 3.2955807250216763e-12,
 4.366230404236119e-12,
 2.100552875389368e-12]

In [61]:
# For F defined on A2, we can restrict our bases to A2

# choose a nonorthonormal-to-A2-vector e.g. e1
v1 = P(e1, A2)
# contract the parallel part
v2 = v1 | A2

frame = multi_frame([v1, v2])
r_frame = reci_frame([v1, v2])
dF = derivative(F, X, alg, frame=frame, r_frame=r_frame)
dFs[-1], dF

(0.000596 + -0.0025 𝐞₁ + -0.00118 𝐞₂ + -0.000636 𝐞₃ + -0.00233 𝐞₄ + -0.00031 𝐞₁₂ + 0.000142 𝐞₁₃ + 0.00011 𝐞₁₄ + 0.000146 𝐞₂₃ + 0.000341 𝐞₂₄ + -0.000105 𝐞₃₄ + -2.84e-12 𝐞₁₂₃ + 1.21e-12 𝐞₁₂₄ + -2.37e-12 𝐞₁₃₄ + -2.99e-12 𝐞₂₃₄ + -2.05e-13 𝐞₁₂₃₄,
 0.000596 + -0.0025 𝐞₁ + -0.00118 𝐞₂ + -0.000636 𝐞₃ + -0.00233 𝐞₄ + -0.00031 𝐞₁₂ + 0.000142 𝐞₁₃ + 0.00011 𝐞₁₄ + 0.000146 𝐞₂₃ + 0.000341 𝐞₂₄ + -0.000105 𝐞₃₄ + -1.15e-11 𝐞₁₂₃ + 9.25e-12 𝐞₁₂₄ + -8.85e-12 𝐞₁₃₄ + -1.64e-11 𝐞₂₃₄ + 2.85e-11 𝐞₁₂₃₄)

In [62]:
#2.18
# double differentials is and ill-conditioned computation
# Always replace d_F_ with dF in computation
# But we can choose a large h to see the equivalence
h = 1e-2 # you can modify h to see how ill it is
F_ = lambda A: differential(F, X, A)
d_F_ = derivative(F_, A, alg, frame=frame, r_frame=r_frame)
# dF = derivative(F , X, alg, frame=frame, r_frame=r_frame)
max_diff(d_F_, dF), d_F_, dF

(2.8805440020857944e-11,
 0.000596 + -0.0025 𝐞₁ + -0.00118 𝐞₂ + -0.000636 𝐞₃ + -0.00233 𝐞₄ + -0.00031 𝐞₁₂ + 0.000142 𝐞₁₃ + 0.00011 𝐞₁₄ + 0.000146 𝐞₂₃ + 0.000341 𝐞₂₄ + -0.000105 𝐞₃₄ + 2.32e-12 𝐞₁₂₃ + 2.17e-12 𝐞₁₂₄ + -9.85e-14 𝐞₁₃₄ + 1.4e-12 𝐞₂₃₄ + -2.57e-13 𝐞₁₂₃₄,
 0.000596 + -0.0025 𝐞₁ + -0.00118 𝐞₂ + -0.000636 𝐞₃ + -0.00233 𝐞₄ + -0.00031 𝐞₁₂ + 0.000142 𝐞₁₃ + 0.00011 𝐞₁₄ + 0.000146 𝐞₂₃ + 0.000341 𝐞₂₄ + -0.000105 𝐞₃₄ + -1.15e-11 𝐞₁₂₃ + 9.25e-12 𝐞₁₂₄ + -8.85e-12 𝐞₁₃₄ + -1.64e-11 𝐞₂₃₄ + 2.85e-11 𝐞₁₂₃₄)

In [63]:
# 2.19
h = 1e-5
def adjoint1(F, X, A, alg: Algebra, frame=None, r_frame=None):
    if not frame:
        frame = multi_frame(alg.frame)
        r_frame = reci_frame(alg.frame)
    _F = 0    
    for v, r in zip(frame, r_frame):
        _F += r * alg.sp(differential(F, X, v), A)
    return _F

# or
def adjoint(F, X, A, alg: Algebra, frame=None, r_frame=None):
    _F = derivative(lambda X: alg.sp(F(X), A), X, alg, frame=frame, r_frame=r_frame)
    return _F

_F = adjoint(F, X, A, alg)
_F1 = adjoint1(F, X, A, alg)

# The algebraic part always determine the order of operations.
# A scalar part's order is arbitrary.
F_BA = differential(F, X, B).sp(A)
F_BA1 = differential(lambda X: F(X).sp(A), X, B)

B.sp(_F), B.sp(_F1), F_BA, F_BA1

(-2.1e-05, -2.1e-05, -2.1e-05, -2.1e-05)

In [64]:
# span of F
vecs = random_r_vectors(alg.d, alg)
frame = multi_frame(vecs)
r_frame = reci_frame(vecs)
F(X), sum(F(X).sp(A) * Ar for A, Ar in zip(frame, r_frame))

(-0.000149 + -0.000243 𝐞₁ + -0.000115 𝐞₂ + -6.17e-05 𝐞₃ + -0.000226 𝐞₄ + -3.01e-05 𝐞₁₂ + 1.38e-05 𝐞₁₃ + 1.07e-05 𝐞₁₄ + 1.42e-05 𝐞₂₃ + 3.31e-05 𝐞₂₄ + -1.02e-05 𝐞₃₄ + -2.71e-20 𝐞₁₂₃ + -2.71e-20 𝐞₁₃₄ + -4.4e-20 𝐞₂₃₄ + -3.39e-21 𝐞₁₂₃₄,
 -0.000149 + -0.000243 𝐞₁ + -0.000115 𝐞₂ + -6.17e-05 𝐞₃ + -0.000226 𝐞₄ + -3.01e-05 𝐞₁₂ + 1.38e-05 𝐞₁₃ + 1.07e-05 𝐞₁₄ + 1.42e-05 𝐞₂₃ + 3.31e-05 𝐞₂₄ + -1.02e-05 𝐞₃₄ + -2.71e-20 𝐞₁₂₃ + -3.31e-35 𝐞₁₂₄ + -2.71e-20 𝐞₁₃₄ + -4.4e-20 𝐞₂₃₄ + -3.39e-21 𝐞₁₂₃₄)

In [65]:
# These are not equal
for A, Ar in zip(frame[:4], r_frame):
    print(adjoint(G, X, A, alg) * Ar, derivative(G, X, alg).sp(A) * Ar, sep='\n')

7.57 + 0.0656 𝐞₁ + -8.11 𝐞₂ + 1.56 𝐞₃ + 4.18 𝐞₄ + 2.16 𝐞₁₂ + 7.01 𝐞₁₃ + 6.76 𝐞₁₄ + 1.99 𝐞₂₃ + 3.11 𝐞₂₄ + 8.62 𝐞₃₄ + 2.9 𝐞₁₂₃ + 4.05 𝐞₁₂₄ + 6.25 𝐞₁₃₄ + 5.49 𝐞₂₃₄ + 2.45 𝐞₁₂₃₄
1.04e+02
-3.01 + -4.41 𝐞₁ + -5.5 𝐞₂ + 13.1 𝐞₃ + -1.59 𝐞₄ + 9.98 𝐞₁₂ + 16.4 𝐞₁₃ + -3.43 𝐞₁₄ + 5.19 𝐞₂₃ + -4.51 𝐞₂₄ + -13.9 𝐞₃₄ + 1.5 𝐞₁₂₃ + 2.46 𝐞₁₂₄ + -21.5 𝐞₁₃₄ + 1.9 𝐞₂₃₄ + -1.95 𝐞₁₂₃₄
2.65 𝐞₁ + -5.85 𝐞₂ + 0.161 𝐞₃ + 7.63 𝐞₄
3.87 + 6.36 𝐞₁ + -2.38 𝐞₂ + 1.38 𝐞₃ + 7.8 𝐞₄ + -7.95 𝐞₁₂ + -4.88 𝐞₁₃ + 4.81 𝐞₁₄ + -2.35 𝐞₂₃ + 4.48 𝐞₂₄ + 7.44 𝐞₃₄ + 1.56 𝐞₁₂₃ + 3.71 𝐞₁₂₄ + 14.2 𝐞₁₃₄ + -0.194 𝐞₂₃₄ + -0.649 𝐞₁₂₃₄
7.96 𝐞₁ + -22.3 𝐞₂ + 17.4 𝐞₃ + 17.4 𝐞₄
-1.87 + 4.24 𝐞₁ + 4.61 𝐞₂ + 6.12 𝐞₃ + -1.18 𝐞₄ + 0.307 𝐞₁₂ + 0.31 𝐞₁₃ + -6.49 𝐞₁₄ + -1.76 𝐞₂₃ + -2.13 𝐞₂₄ + -1.77 𝐞₃₄ + -3.35 𝐞₁₂₃ + 0.37 𝐞₁₂₄ + 4.85 𝐞₁₃₄ + 4.69 𝐞₂₃₄ + 0.346 𝐞₁₂₃₄
-2.99 𝐞₁ + -0.835 𝐞₂ + -1.61 𝐞₃ + -0.0242 𝐞₄


In [66]:
# But after summing
(
    derivative(G, X, alg), 
    sum(adjoint(G, X, A, alg) * Ar for A, Ar in zip(frame, r_frame)),
    sum(derivative(G, X, alg).sp(A) * Ar for A, Ar in zip(frame, r_frame)),
)

(40.0 + -8.76 𝐞₁ + -41.2 𝐞₂ + 10.2 𝐞₃ + 20.3 𝐞₄ + -11.0 𝐞₁₂ + 10.5 𝐞₁₃ + 14.3 𝐞₁₄ + 5.75 𝐞₂₃ + 11.1 𝐞₂₄ + 15.3 𝐞₃₄ + -1.23 𝐞₁₂₃ + 16.6 𝐞₁₂₄ + 40.8 𝐞₁₃₄ + 14.9 𝐞₂₃₄ + 5.99 𝐞₁₂₃₄,
 40.0 + -8.76 𝐞₁ + -41.2 𝐞₂ + 10.2 𝐞₃ + 20.3 𝐞₄ + -11.0 𝐞₁₂ + 10.5 𝐞₁₃ + 14.3 𝐞₁₄ + 5.75 𝐞₂₃ + 11.1 𝐞₂₄ + 15.3 𝐞₃₄ + -1.23 𝐞₁₂₃ + 16.6 𝐞₁₂₄ + 40.8 𝐞₁₃₄ + 14.9 𝐞₂₃₄ + 5.99 𝐞₁₂₃₄,
 40.0 + -8.76 𝐞₁ + -41.2 𝐞₂ + 10.2 𝐞₃ + 20.3 𝐞₄ + -11.0 𝐞₁₂ + 10.5 𝐞₁₃ + 14.3 𝐞₁₄ + 5.75 𝐞₂₃ + 11.1 𝐞₂₄ + 15.3 𝐞₃₄ + -1.23 𝐞₁₂₃ + 16.6 𝐞₁₂₄ + 40.8 𝐞₁₃₄ + 14.9 𝐞₂₃₄ + 5.99 𝐞₁₂₃₄)

In [67]:
#2.20 F defined on A2
max_diff(P(_F, A2), _F)

0.0010093927008320612

In [68]:
#2.21
A = random_multivector(alg)
_F = adjoint(F, X, A, alg)
[max_diff(_F.grade(r), derivative(lambda X: F(X).sp(A), X, alg, r)) for r in range(0, 5)]

[0.0, 0.0, 0.0, 0.0, 0.0]

In [69]:
#2.24
r = 2
_sum = derivative(lambda X: F(X), X, alg) + derivative(lambda X: G(X), X, alg)
max_diff(derivative(lambda X: F(X) + G(X), X, alg), _sum)

2.4365931494685356e-10

In [70]:
#2.25a
h = 1e-6
_prod = derivative(lambda X: F(X), X, alg)*G(X) + derivative(lambda Y:F(X)*G(Y), X, alg)
max_diff(derivative(lambda X: F(X)*G(X), X, alg), _prod)

3.110059085265071e-12

In [71]:
#2.25b Hitzer regard this as a definition. But we shall see it's not! The over dot just means "at once".
# r-derivative
grade = 2
frame = r_vector_frame(alg.frame, grade)
r_frame = r_vector_frame(reciprocal(alg.frame), grade, reverse=True)
FdG = 0
Fd = 0
dG = 0
for v, r in zip(frame, r_frame):
    FdG += differential(lambda X: F(X)*r*G(X), X, v) # the over dots made clear using lambda function
    Fd += differential(F, X, v) * r
    dG += r * differential(G, X, v)
max_diff(FdG, (Fd * G(X)) + (F(X)*dG))

1.478029504342615e-12

In [72]:
#2.25b derivative on entire frame
frame = multi_frame(alg.frame)
r_frame = reci_frame(alg.frame)
FdG = 0
Fd = 0
dG = 0
for v, r in zip(frame, r_frame):
    FdG += differential(lambda X: F(X)*r*G(X), X, v)
    Fd += differential(F, X, v) * r
    dG += r * differential(G, X, v)
max_diff(FdG, (Fd * G(X)) + (F(X)*dG))

1.8415546865213628e-12

In [73]:
#2.26 r-derivative
h = 1e-4
r = 2
max_diff(
derivative(GF, X, alg, r), 
derivative(lambda Y: differential(G, F(X), F(Y)), X, alg, r)
)

3.601871867431328e-08

In [74]:
#2.26 derivative
h = 1e-4
r = None
max_diff(
derivative(GF, X, alg, r), 
derivative(lambda Y: differential(G, F(X), F(Y)), X, alg, r)
)

6.81925484968815e-08

In [75]:
#2.28a
(
    max_diff(P(A, A2), differential(lambda X: P(X, A2), X, A)),
    max_diff(P(A, A2), derivative(lambda X: P(X, A2).sp(A), X, alg))
)

(6.693257059708912e-14, 4.128641872824801e-14)

In [76]:
#2.28b
max_diff(P(A, A2).reverse(), differential(lambda X: P(X.reverse(), A2), X, A))

5.1084136920565015e-14

In [77]:
#2.29 d of n dim spade is 2^n
h = 1e-5
A3 = random_r_blade(3, alg)
(
derivative(lambda X: X, X, alg), #4d
derivative(lambda X: P(X, A3), X, alg), #3d
derivative(lambda X: P(X, A2), X, alg) #2d
)

(16.0,
 -1.03 + 4.55e-13 𝐞₁₂ + 4.55e-13 𝐞₁₃ + -2.96e-12 𝐞₂₃ + -1.19e-12 𝐞₂₄ + -6.82e-13 𝐞₃₄,
 -0.0784 + -9.95e-14 𝐞₁₂ + 8.38e-13 𝐞₁₃ + 3.27e-13 𝐞₁₄ + -5.12e-13 𝐞₂₃ + 9.95e-14 𝐞₂₄ + -1.14e-13 𝐞₃₄ + 7.11e-14 𝐞₁₂₃₄)

In [78]:
#2.30
max_diff(derivative(lambda X: X.sp(X.reverse()), X, alg), 2*X.reverse())

1.5609513681624776e-11

In [79]:
#2.31
k=6
max_diff(differential(lambda X: X**k, X, A), sum((X**l)*A*(X**(k-1-l)) for l in range(k)))

1.4291799743659794e-08

In [80]:
#2.32 To allow different signature, we consider norm square instead
def normsq(X):
    return X.sp(X.reverse())[0]

k=3    
max_diff(derivative(lambda X: normsq(X) ** (k), X, alg), 2*k*normsq(X)**(k-1)*X.reverse())

1.6145271786172088e-09

In [81]:
#2.33
max_diff(derivative(lambda X: np.log(np.abs(normsq(X))), X, alg), 2 * X.reverse()/normsq(X))

6.532858698449218e-10

In [82]:
#2.30 var
for n in range(7):
    alg = Algebra(n)
    X = random_multivector(alg)
    dXXr = derivative(lambda X: X * X.reverse(), X, alg)
    Xr = X.reverse()
    ratio = np.round(dXXr[0]/Xr[0])
    print(ratio, max_diff(dXXr, ratio*Xr)) 

2.0 6.838529742481114e-12
4.0 8.852474309151148e-12
6.0 1.6653345369377348e-11
8.0 6.391243090320131e-11
12.0 1.4205880916051683e-10
24.0 2.433750978525495e-10
56.0 3.823075189757219e-10


Compare with 2.2.30. The derivative is also an integer multiple of $\tilde{X}$, yet a complex one:
$$ \partial  X\tilde{X} = \.{\partial} \.{X} \tilde{X} + \.{\partial} X \.{\tilde{X}} = 2^n \tilde{X} + {\gamma}_n \tilde{X}$$
where ${\gamma}_n$ are suggests to be [1, 2, 2, 0, -4, -8, -8] for $0\leq n \leq 6$ from the above and below numerical tests:

In [83]:
for n in range(1,7):
    alg = Algebra(n)
    X = random_multivector(alg)
    dXXr = derivative(lambda Y: X*Y.reverse(), X, alg)
    print(n, dXXr[0]/X[0])

1 1.9999999999881675
2 2.0000000000152474
3 1.8324966037582148e-11
4 -4.000000000069896
5 -7.999999997017601
6 -7.999999999849673


$\partial X^2$ is conversely neither a multiple of $X$ nor $\tilde{X}$.
But if we separate scalars or pseudoscalars from other grades, another integer multiple emerges. 
Try proving it!

In [84]:
# 2.30 var2
for n in range(2,7):
    alg = Algebra(n)
    X = random_multivector(alg)
    dXX = derivative(lambda X: X**2, X, alg)
    ratios = np.round(dXX[0]/X[0])
    ratiov = np.round(dXX[1]/X[1])
    ratiops = np.round(dXX[2**n-1]/X[2**n-1])
    print(ratios, ratiov, ratiops, max_diff(dXX[1:2**n-1], ratiov*(X[1:2**n-1]))) 

8.0 4.0 4.0 1.6326495710927702e-11
16.0 8.0 16.0 1.4634515821398963e-11
32.0 16.0 16.0 6.066258606551855e-11
64.0 32.0 64.0 2.986730862630793e-10
128.0 64.0 64.0 4.817621857000631e-10


In [85]:
#2.37a
from scipy.special import comb
for n, r in [(5,2), (6,2), (1,1), (1,0), (2,0)]:
    alg = Algebra(n)
    X = random_multivector(alg)
    print(derivative(lambda X: X, X, alg, r), comb(n, r, exact=True))

10.0 10
15.0 15
1.0 1
1.0 1
1.0 1


In [86]:
#2.38a
def gamma(n, r, s):
    gamma = 0
    K = int(np.round((r + s - abs(r - s))/2))
    for k in range(0, K+1):
        gamma += (-1)**(r*s-k)*comb(r, k, exact=True)*comb(n-r, s-k, exact=True)
    return gamma

for n, r, s in [(5,2,1), (6,2,3), (1,1,1), (1,0,1), (2,0,1)]:
    alg = Algebra(n)
    X = random_multivector(alg)
    Ar = random_multivector(alg).grade(r)
    print(max_diff(derivative(lambda X: Ar*X, X, alg, s), gamma(n, r, s)*Ar)) 

2.0550006141206723e-11
6.520206596860589e-11
2.994271497414047e-13
8.344103186175289e-12
2.396083331746013e-12


In [87]:
#2.38b
n = 5
s = 2
alg = Algebra(n)
X = random_multivector(alg)
A = random_multivector(alg)
max_diff(derivative(lambda X: A*X, X, alg, s), sum(gamma(n, r, s)*A.grade(r) for r in range(0, n+1)))


1.0440914799403345e-10

Construct $\gamma_n$ from $\Gamma_s^r$

In [88]:
#2.38a reverse version
def gamma_rev(n, r, s):
    gamma = 0
    K = int(np.round((r + s - abs(r - s))/2))
    for k in range(0, K+1):
        gamma += (-1)**(r*s-k+s*(s-1)/2)*comb(r, k, exact=True)*comb(n-r, s-k, exact=True)
    return gamma

for n, r, s in [(5,2,1), (6,2,3), (1,1,1), (1,0,1), (2,0,1)]:
    alg = Algebra(n)
    X = random_multivector(alg)
    Ar = random_multivector(alg).grade(r)
    dAXr = derivative(lambda X: Ar*X.reverse(), X, alg, s)
    print(max_diff(dAXr, gamma_rev(n, r, s)*Ar)) 

5.843647787884265e-11
4.987432689063098e-11
7.437050975056536e-12
5.432321259490891e-13
2.2311041902867146e-12


In [89]:
for n in range(7):
    r_parts = np.zeros(n+1)
    for s in range(0, n+1):
        for r in range(0, n+1):
            r_parts[r] += (-1)**(r*(r-1)/2)*gamma_rev(n, r, s) # one more reversion
    print(r_parts)

[1.]
[2. 2.]
[2. 2. 2.]
[0. 0. 0. 0.]
[-4. -4. -4. -4. -4.]
[-8. -8. -8. -8. -8. -8.]
[-8. -8. -8. -8. -8. -8. -8.]


The coefficients of each grade match due to the reversion-symmetry of gamma_rev.
Thus we get $\gamma_n$ by fixing any r:

In [90]:
def gamma_n(n):
    return sum(gamma_rev(n, 0, s) for s in range(0, n+1))
[gamma_n(n) for n in range(7)]

[1.0, 2.0, 2.0, 0.0, -4.0, -8.0, -8.0]

In [91]:
#3.1a dAG and ddGA are equivalent according to 2.18
h = 1e-3
alg = Algebra(2)
A = random_multivector(alg)
B = random_multivector(alg)
G = lambda X: (X+B) ** 2

dAG = derivative(lambda A: G(A^B), A, alg)
ddGA = derivative(lambda A_: differential(G, A^B, A_^B), A, alg)

max_diff(dAG, ddGA), dAG, ddGA

(5.969269523120602e-11,
 2.87 + 1.2 𝐞₁ + -0.107 𝐞₂ + 0.0657 𝐞₁₂,
 2.87 + 1.2 𝐞₁ + -0.107 𝐞₂ + 0.0657 𝐞₁₂)

In [92]:
#3.1b a headache...
h = 1e-2 # sufficiently large to see the equivalence

ddGAB = derivative(lambda B: derivative(lambda A: G(A^B), A, alg), B, alg)
dAdBG = (
    derivative(lambda B_: derivative(lambda A_: differential(G, A^B, A_^B_), A, alg), B, alg) +
    derivative(lambda B_: differential(lambda X: derivative(lambda A_: differential(G, X, A_^B), A, alg), A^B, A^B_), B, alg)
    )
max_diff(ddGAB, dAdBG), ddGAB, dAdBG

(1.1572612201860011e-08,
 4.81 + 9.08 𝐞₁ + 1.95 𝐞₂ + -3.12 𝐞₁₂,
 4.81 + 9.08 𝐞₁ + 1.95 𝐞₂ + -3.12 𝐞₁₂)

In [93]:
#3.2b
h = 1e-5
for r in range(1, 7):
    alg = Algebra(r)
    G = lambda X: (X.grade(r)+B) ** 2
    frame = r_vector_frame(alg.frame, r)
    r_frame = r_vector_frame(reciprocal(alg.frame), r, reverse=True)
    for s in range(r):
        A = random_multivector(alg).grade(s)
        B = random_multivector(alg).grade(r-s)

        dAG = derivative(lambda A: G(A^B), A, alg, grade=s)
        BdG = 0
        for v, vr in zip(frame, r_frame):
            BdG += inner(B, vr) * (differential(G, A^B, v))
        assert_diff(dAG, BdG, 1e-8)
        

In [94]:
#3.2c Somehow the pseudoscalars are more ill conditioned, let me know if you figure out
h = 1e-2
for r in range(2,5):
    alg = Algebra(r)
    G = lambda X: (X.grade(r)+B) ** 2
    frame = r_vector_frame(alg.frame, r)
    r_frame = r_vector_frame(reciprocal(alg.frame), r, reverse=True)
    for s in range(r):
        A = random_multivector(alg).grade(s)
        B = random_multivector(alg).grade(r-s)

        dAG = lambda B: derivative(lambda A: G(A^B), A, alg, grade=s)
        dBdAG = derivative(dAG, B, alg, grade=r-s)
        BdG = 0
        for v, vr in zip(frame, r_frame):
            for u, ur in zip(frame, r_frame):
                BdG += inner(A, u) * inner(B, vr) * (differential(lambda X: differential(G, X, ur), A^B, v))
        BdG *= (-1)**(s*(r-s))
        BdG += comb(r, r-s)*derivative(G, A^B, alg, grade=r)

        print(dBdAG, BdG, sep="\n")

2.9 𝐞₁₂
2.08 𝐞₁₂
1.19 + -3.03 𝐞₁₂
1.19 + -3.03 𝐞₁₂
2.06 𝐞₁₂₃
1.9 𝐞₁₂₃
6.93e-12 + -0.495 𝐞₁ + 0.0866 𝐞₂ + 0.489 𝐞₃ + 5.98 𝐞₁₂ + 0.407 𝐞₁₃ + 5.84 𝐞₂₃ + 11.3 𝐞₁₂₃
-0.495 𝐞₁ + 0.0866 𝐞₂ + 0.489 𝐞₃ + 5.98 𝐞₁₂ + 0.407 𝐞₁₃ + 5.84 𝐞₂₃ + 11.3 𝐞₁₂₃
5.46 𝐞₁ + 1.73 𝐞₂ + 2.94 𝐞₃ + 1.92 𝐞₁₂₃
5.46 𝐞₁ + 1.73 𝐞₂ + 2.94 𝐞₃ + 1.92 𝐞₁₂₃
2.45 𝐞₁₂₃₄
1.96 𝐞₁₂₃₄
0.608 𝐞₁₂ + 0.806 𝐞₁₃ + -0.382 𝐞₁₄ + 0.713 𝐞₂₃ + 0.243 𝐞₂₄ + 0.77 𝐞₃₄ + -12.8 𝐞₁₂₃₄
0.608 𝐞₁₂ + 0.806 𝐞₁₃ + -0.382 𝐞₁₄ + 0.713 𝐞₂₃ + 0.243 𝐞₂₄ + 0.77 𝐞₃₄ + -12.8 𝐞₁₂₃₄
-3.57 + 7.48 𝐞₁₂ + 11.2 𝐞₁₃ + 1.49 𝐞₁₄ + 9.78 𝐞₂₃ + 10.1 𝐞₂₄ + 4.73 𝐞₃₄ + 11.3 𝐞₁₂₃₄
-3.57 + 7.48 𝐞₁₂ + 11.2 𝐞₁₃ + 1.49 𝐞₁₄ + 9.78 𝐞₂₃ + 10.1 𝐞₂₄ + 4.73 𝐞₃₄ + 11.3 𝐞₁₂₃₄
1.31 𝐞₁₂ + -1.23 𝐞₁₃ + -1.85 𝐞₁₄ + 1.48 𝐞₂₃ + 0.643 𝐞₂₄ + 1.48 𝐞₃₄ + -0.263 𝐞₁₂₃₄
1.31 𝐞₁₂ + -1.23 𝐞₁₃ + -1.85 𝐞₁₄ + 1.48 𝐞₂₃ + 0.643 𝐞₂₄ + 1.48 𝐞₃₄ + -0.263 𝐞₁₂₃₄


In [95]:
#3.4
h = 1e-3
alg = Algebra(r)
C = random_multivector(alg)
D = random_multivector(alg)
L = lambda X: C * X * D
dAL = lambda B: derivative(lambda A: L(A^B), A, alg, grade=s)
dBdAL = derivative(dAL, B, alg, grade=r-s)
max_diff(dBdAL, comb(r, r-s) * derivative(L, A^B, alg, grade=r))

7.550404745870765e-11

In [96]:
def multi_differential(F, X, vectors):
    from functools import reduce
    
    def apply_differential(current_func, vector):
        return lambda X: differential(current_func, X, vector)
    
    # Reduce applies the function cumulatively to the items of iterable, from left to right,
    # so as to reduce the iterable to a single value.
    return reduce(apply_differential, vectors, F)(X)

In [97]:
def multi_derivative(F, vectors):
    def apply_derivative(current_func, V):
        assert_simple(V)
        return lambda X: derivative(lambda Y: current_func(Y ^ X), V, alg, grade=V.grades[0])
    
    return reduce(apply_derivative, vectors, F)(1)

# Excercise: use frame and differential to split the derivatives. Iterate over multiindexes sounds a good idea

In [98]:
#3.5
from math import factorial
h = 1e-2
alg = Algebra(1,3)
C = random_multivector(alg)
D = random_multivector(alg)
L = lambda X: C * X * D
r = 3
n = len(alg.frame)
frame = alg.frame
r_frame = reciprocal(alg.frame)
vectors = random_r_vectors(r, alg)
X = wedge(vectors)

drL = multi_derivative(L, vectors) 
drL, factorial(r)*derivative(L, X, alg, grade=r)


(-1.66 + 3.51 𝐞₁ + 14.2 𝐞₂ + 24.2 𝐞₃ + -11.4 𝐞₄ + 7.93 𝐞₁₂ + -9.62 𝐞₁₃ + 16.4 𝐞₁₄ + -9.35 𝐞₂₃ + 1.71 𝐞₂₄ + 7.58 𝐞₃₄ + 20.3 𝐞₁₂₃ + -5.13 𝐞₁₂₄ + 12.5 𝐞₁₃₄ + 8.38 𝐞₂₃₄ + -11.9 𝐞₁₂₃₄,
 -1.66 + 3.51 𝐞₁ + 14.2 𝐞₂ + 24.2 𝐞₃ + -11.4 𝐞₄ + 7.93 𝐞₁₂ + -9.62 𝐞₁₃ + 16.4 𝐞₁₄ + -9.35 𝐞₂₃ + 1.71 𝐞₂₄ + 7.58 𝐞₃₄ + 20.3 𝐞₁₂₃ + -5.13 𝐞₁₂₄ + 12.5 𝐞₁₃₄ + 8.38 𝐞₂₃₄ + -11.9 𝐞₁₂₃₄)

In [99]:
# We can also expand the vector derivatives. And a linear function equals to its differential.
drL = 0
r = 3
for v, vr in zip(frame, r_frame):
    for u, ur in zip(frame, r_frame):
        for w, wr in zip(frame, r_frame):
            drL += vr * u * w * L(wr^ur^v)
drL

-1.66 + 3.51 𝐞₁ + 14.2 𝐞₂ + 24.2 𝐞₃ + -11.4 𝐞₄ + 7.93 𝐞₁₂ + -9.62 𝐞₁₃ + 16.4 𝐞₁₄ + -9.35 𝐞₂₃ + 1.71 𝐞₂₄ + 7.58 𝐞₃₄ + 20.3 𝐞₁₂₃ + -5.13 𝐞₁₂₄ + 12.5 𝐞₁₃₄ + 8.38 𝐞₂₃₄ + -11.9 𝐞₁₂₃₄

In [100]:
# A cleaner version of above
from itertools import permutations
drL = 0
for base_vectors, reci_vectors in zip(permutations(frame, r), permutations(r_frame, r)):
    drL += wedge(base_vectors[::-1]) * L(wedge(reci_vectors))
drL

-1.66 + 3.51 𝐞₁ + 14.2 𝐞₂ + 24.2 𝐞₃ + -11.4 𝐞₄ + 7.93 𝐞₁₂ + -9.62 𝐞₁₃ + 16.4 𝐞₁₄ + -9.35 𝐞₂₃ + 1.71 𝐞₂₄ + 7.58 𝐞₃₄ + 20.3 𝐞₁₂₃ + -5.13 𝐞₁₂₄ + 12.5 𝐞₁₃₄ + 8.38 𝐞₂₃₄ + -11.9 𝐞₁₂₃₄

In [101]:
from itertools import product
def vectors_partial(F, vectors, directions):
    r = len(vectors)
    drF = 0
    for offset in product([1,-1], repeat=r):
        offset = np.array(offset)
        coef = np.prod(offset)
        offset = offset * h
        drF += 1/(2*h)**r * coef * F([a+v*d for a, v, d in zip(vectors, directions, offset)])
    return drF

In [102]:
#3.6 simplicial derivative
from itertools import permutations
h = 1e-2
def simplicial_derivative(F, vectors, alg):
    frame = alg.frame
    r_frame = reciprocal(alg.frame)
    drF = 0
    r = len(vectors)
    for base_vectors, reci_vectors in zip(permutations(frame, r), permutations(r_frame, r)):
        drF += wedge(base_vectors[::-1]) * vectors_partial(F, vectors, reci_vectors)
    return (1/factorial(r)) * drF

simplicial_derivative(lambda vectors: L(wedge(vectors)), vectors, alg) * factorial(r)

-1.66 + 3.51 𝐞₁ + 14.2 𝐞₂ + 24.2 𝐞₃ + -11.4 𝐞₄ + 7.93 𝐞₁₂ + -9.62 𝐞₁₃ + 16.4 𝐞₁₄ + -9.35 𝐞₂₃ + 1.71 𝐞₂₄ + 7.58 𝐞₃₄ + 20.3 𝐞₁₂₃ + -5.13 𝐞₁₂₄ + 12.5 𝐞₁₃₄ + 8.38 𝐞₂₃₄ + -11.9 𝐞₁₂₃₄

In [103]:
#3.7a
h = 1e-2
for n, r in [(5,5),(5,4),(5,3),(6,5),(6,4),(6,3)]:
    alg = Algebra(n)
    vectors = random_r_vectors(r, alg)
    print(simplicial_derivative(lambda vectors: wedge(vectors), vectors, alg))

1.0
5.0 + -1.46e-11 𝐞₁₃ + 1.21e-12 𝐞₁₅
10.0 + -3.03e-13 𝐞₁₂ + 1.89e-14 𝐞₁₃ + -1.52e-13 𝐞₁₅ + 1.71e-13 𝐞₂₃ + -5.68e-14 𝐞₂₄ + -7.58e-14 𝐞₂₅ + -9.47e-14 𝐞₃₄ + -3.79e-14 𝐞₃₅ + 3.79e-14 𝐞₄₅
6.0 + 7.76e-12 𝐞₁₂ + 6.55e-11 𝐞₁₃ + -7.76e-12 𝐞₁₄ + -4.66e-11 𝐞₁₅ + -3.88e-11 𝐞₁₆ + 2.67e-11 𝐞₂₃ + -3.88e-11 𝐞₂₄ + -9.75e-11 𝐞₃₄ + 1.55e-11 𝐞₃₅ + 9.7e-13 𝐞₃₆ + 7.76e-12 𝐞₄₅ + 3.88e-12 𝐞₄₆
15.0 + -9.09e-12 𝐞₁₂ + 1.7e-11 𝐞₁₃ + 7.28e-12 𝐞₁₄ + -1.82e-12 𝐞₁₅ + 1.21e-11 𝐞₁₆ + 3.88e-11 𝐞₂₃ + 1.49e-11 𝐞₂₄ + 2.55e-11 𝐞₂₅ + 1.7e-11 𝐞₂₆ + 1.38e-10 𝐞₃₄ + 3.88e-11 𝐞₃₅ + -1.46e-11 𝐞₃₆ + 9.09e-13 𝐞₄₅ + -1.5e-10 𝐞₄₆ + -5.34e-11 𝐞₅₆
20.0 + 9.09e-13 𝐞₁₂ + 8.53e-14 𝐞₁₃ + 4.74e-13 𝐞₁₄ + 5.87e-13 𝐞₁₅ + -3.51e-13 𝐞₂₃ + -6.44e-13 𝐞₂₄ + -5.78e-13 𝐞₂₅ + 3.03e-13 𝐞₂₆ + -2.84e-13 𝐞₃₄ + 9.47e-15 𝐞₃₅ + -8.53e-14 𝐞₃₆ + 5.31e-13 𝐞₄₅ + 9.28e-13 𝐞₄₆ + 4.36e-13 𝐞₅₆


In [104]:
#3.7b X^2 is certainly not a linear function. I gave the r-derivatives of X^2 in 2.30 var2
h = 1e-3
for n, r in [(5,5),(5,4),(5,3),(6,5),(6,4),(6,3)]:
    alg = Algebra(n)
    vectors = random_r_vectors(r, alg)
    print(max_diff(simplicial_derivative(lambda vectors: wedge(vectors)**2, vectors, alg), (r+1) * wedge(vectors)))

1.871462351754616e-08
4.314186680121068e-08
1.4175395267324031e-09
0.00013632283500264464
9.82385192360713e-08
2.777204272108591e-08


In [105]:
#3.8 Just a ratio of factorials
r, s = 7, 4
factorial(r+s) / (factorial(r) * factorial(s)), comb(r+s, s)

(330.0, 330.0)

In [106]:
#3.9
n = 5
r = 4
h = 1e-5
alg = Algebra(n)
A_list = [random_multivector(alg) for _ in range(r)]
B_list = [random_multivector(alg) for _ in range(r)]
vectors = random_r_vectors(r, alg)
Fr = lambda vectors: sum(A_list[i] * vectors[i] * B_list[i] for i in range(r))

akpkFp = 0
akpkFn = 0
for i in range(r):
    akpkFp += Fr(vectors[:i] +[vectors[i]+vectors[i]*h] + vectors[i+1:])
    akpkFn += Fr(vectors[:i] +[vectors[i]-vectors[i]*h] + vectors[i+1:])

akpkFp *= 1/(2*h)
akpkFn *= 1/(2*h)
akpkF = akpkFp - akpkFn
akpkF, Fr(vectors)


(-2.2 + -1.48 𝐞₁ + 2.01 𝐞₂ + 1.87 𝐞₃ + -2.28 𝐞₄ + -5.94 𝐞₅ + -2.57 𝐞₁₂ + 0.662 𝐞₁₃ + -2.85 𝐞₁₄ + -4.08 𝐞₁₅ + 3.82 𝐞₂₃ + 17.1 𝐞₂₄ + 1.32 𝐞₂₅ + 2.43 𝐞₃₄ + 0.421 𝐞₃₅ + 2.5 𝐞₄₅ + 7.24 𝐞₁₂₃ + 11.9 𝐞₁₂₄ + 1.63 𝐞₁₂₅ + 5.49 𝐞₁₃₄ + 0.581 𝐞₁₃₅ + -0.405 𝐞₁₄₅ + 7.0 𝐞₂₃₄ + 7.48 𝐞₂₃₅ + 9.46 𝐞₂₄₅ + 1.88 𝐞₃₄₅ + 9.39 𝐞₁₂₃₄ + 13.4 𝐞₁₂₃₅ + 8.59 𝐞₁₂₄₅ + 7.85 𝐞₁₃₄₅ + 6.0 𝐞₂₃₄₅ + 10.4 𝐞₁₂₃₄₅,
 -2.2 + -1.48 𝐞₁ + 2.01 𝐞₂ + 1.87 𝐞₃ + -2.28 𝐞₄ + -5.94 𝐞₅ + -2.57 𝐞₁₂ + 0.662 𝐞₁₃ + -2.85 𝐞₁₄ + -4.08 𝐞₁₅ + 3.82 𝐞₂₃ + 17.1 𝐞₂₄ + 1.32 𝐞₂₅ + 2.43 𝐞₃₄ + 0.421 𝐞₃₅ + 2.5 𝐞₄₅ + 7.24 𝐞₁₂₃ + 11.9 𝐞₁₂₄ + 1.63 𝐞₁₂₅ + 5.49 𝐞₁₃₄ + 0.581 𝐞₁₃₅ + -0.405 𝐞₁₄₅ + 7.0 𝐞₂₃₄ + 7.48 𝐞₂₃₅ + 9.46 𝐞₂₄₅ + 1.88 𝐞₃₄₅ + 9.39 𝐞₁₂₃₄ + 13.4 𝐞₁₂₃₅ + 8.59 𝐞₁₂₄₅ + 7.85 𝐞₁₃₄₅ + 6.0 𝐞₂₃₄₅ + 10.4 𝐞₁₂₃₄₅)

In [107]:
#3.10
h = 1e-2

def skew_symmetrizer(F, vectors, alg):
    frame = alg.frame
    r_frame = reciprocal(alg.frame)
    drF = 0
    r = len(vectors)
    Ar = wedge(vectors)
    for base_vectors, reci_vectors in zip(permutations(frame, r), permutations(r_frame, r)):
        # If F is linear, vectors cause no difference in vectors_partial. The input is just Ar.
        # And we can allow Ar being any multivector
        drF += (Ar.sp(wedge(base_vectors[::-1]))) * vectors_partial(F, vectors, reci_vectors)
    return (1/factorial(r)) * drF

skew_symmetrizer(Fr, vectors, alg)

6.86e-11 + 5.99e-11 𝐞₁ + 2.84e-11 𝐞₂ + -9.29e-11 𝐞₃ + 8.72e-11 𝐞₄ + 7.17e-11 𝐞₅ + 1.98e-10 𝐞₁₂ + 1.33e-12 𝐞₁₃ + -1.41e-10 𝐞₁₄ + -2.11e-10 𝐞₁₅ + -4.88e-11 𝐞₂₃ + -2.52e-10 𝐞₂₄ + 1.28e-10 𝐞₂₅ + 2.22e-10 𝐞₃₄ + 8.53e-13 𝐞₃₅ + 2.7e-11 𝐞₄₅ + 2.8e-10 𝐞₁₂₃ + -3.7e-10 𝐞₁₂₄ + -5.08e-11 𝐞₁₂₅ + -6.19e-10 𝐞₁₃₄ + 1.09e-11 𝐞₁₃₅ + -2.69e-11 𝐞₁₄₅ + 1.74e-10 𝐞₂₃₄ + 5.14e-10 𝐞₂₃₅ + -9.87e-11 𝐞₂₄₅ + -5.89e-11 𝐞₃₄₅ + 4.82e-10 𝐞₁₂₃₄ + -1.72e-09 𝐞₁₂₃₅ + 3.21e-10 𝐞₁₂₄₅ + 4.52e-10 𝐞₁₃₄₅ + 1.4e-10 𝐞₂₃₄₅ + -1.6e-10 𝐞₁₂₃₄₅

In [108]:
#3.11
h = 1e-2
C = random_multivector(alg)
D = random_multivector(alg)
L = lambda vectors: C * wedge(vectors) * D
skew_symmetrizer(L, vectors, alg), L(vectors)

(-0.31 + -0.345 𝐞₁ + -0.211 𝐞₂ + -0.41 𝐞₃ + -0.113 𝐞₄ + -0.427 𝐞₅ + -0.076 𝐞₁₂ + -0.247 𝐞₁₃ + 0.0418 𝐞₁₄ + -0.421 𝐞₁₅ + 0.186 𝐞₂₃ + -0.149 𝐞₂₄ + -0.396 𝐞₂₅ + -0.0475 𝐞₃₄ + -0.372 𝐞₃₅ + -0.417 𝐞₄₅ + 0.0213 𝐞₁₂₃ + -0.191 𝐞₁₂₄ + -0.148 𝐞₁₂₅ + -0.369 𝐞₁₃₄ + -0.418 𝐞₁₃₅ + -0.021 𝐞₁₄₅ + -0.103 𝐞₂₃₄ + 0.71 𝐞₂₃₅ + -0.164 𝐞₂₄₅ + -0.419 𝐞₃₄₅ + 0.399 𝐞₁₂₃₄ + 0.542 𝐞₁₂₃₅ + -0.0564 𝐞₁₂₄₅ + -0.481 𝐞₁₃₄₅ + -0.457 𝐞₂₃₄₅ + -0.0413 𝐞₁₂₃₄₅,
 -0.31 + -0.345 𝐞₁ + -0.211 𝐞₂ + -0.41 𝐞₃ + -0.113 𝐞₄ + -0.427 𝐞₅ + -0.076 𝐞₁₂ + -0.247 𝐞₁₃ + 0.0418 𝐞₁₄ + -0.421 𝐞₁₅ + 0.186 𝐞₂₃ + -0.149 𝐞₂₄ + -0.396 𝐞₂₅ + -0.0475 𝐞₃₄ + -0.372 𝐞₃₅ + -0.417 𝐞₄₅ + 0.0213 𝐞₁₂₃ + -0.191 𝐞₁₂₄ + -0.148 𝐞₁₂₅ + -0.369 𝐞₁₃₄ + -0.418 𝐞₁₃₅ + -0.021 𝐞₁₄₅ + -0.103 𝐞₂₃₄ + 0.71 𝐞₂₃₅ + -0.164 𝐞₂₄₅ + -0.419 𝐞₃₄₅ + 0.399 𝐞₁₂₃₄ + 0.542 𝐞₁₂₃₅ + -0.0564 𝐞₁₂₄₅ + -0.481 𝐞₁₃₄₅ + -0.457 𝐞₂₃₄₅ + -0.0413 𝐞₁₂₃₄₅)

In [109]:
def blade_split(Ar, alg):
    r = Ar.grades[0]
    projects = [P(e, Ar) for e in alg.frame[:r]]
    wed = wedge(projects)
    ratio = Ar[:][0]/wed[:][0]
    return projects, ratio

In [112]:
#3.12
r = 3
alg = Algebra(5)
locals().update(alg.blades)
Ar = random_r_blade(r, alg)
alpha = lambda vectors: wedge(vectors) | Ar
max_diff(simplicial_derivative(alpha, np.zeros(r), alg), Ar)

5.551115123125783e-17